In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config("spark.ui.port", "0"). \
    enableHiveSupport(). \
    appName("Spark GC Issues"). \
    master("yarn"). \
    config('spark.executor.instances','1'). \
    config('spark.executor.memory','512MB'). \
    config('spark.executor.cores','2'). \
    config('spark.driver.memory','512MB'). \
    config('spark.driver.cores','2'). \
    config('spark.dynamicAllocation.enabled','False'). \
    getOrCreate()

In [ ]:
spark.stop()

In [11]:
spark.sparkContext.applicationId

'application_1745651200635_18582'

In [23]:
 #Disable AQE and Broadcast join

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [31]:
# Create large skewed dataset
from pyspark.sql import functions as F

df1 = spark.range(0, 1_000_000_000).withColumn("key", (F.rand() * 10).cast("int"))  # 100M rows
df2 = spark.range(0, 1_000).withColumnRenamed("id", "key")


In [36]:
 spark.conf.set("spark.sql.shuffle.partitions", "500") 

In [37]:
# Shuffle-heavy skewed join
joined = df1.join(df2, "key")


In [38]:
# Add wide aggregation (forces memory usage)

agg = joined.groupBy("key").count()

In [39]:

agg.show()

+---+---------+
|key|    count|
+---+---------+
|  1|100006020|
|  6| 99999621|
|  3|100011190|
|  2|100000277|
|  4| 99992545|
|  8| 99991080|
|  7| 99975935|
|  0|100008685|
|  9|100010228|
|  5|100004419|
+---+---------+



In [1]:
#Enable GC Logging
#Add the following in Spark config (spark.executor.extraJavaOptions and spark.driver.extraJavaOptions):

-verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -Xloggc:/path/to/gc.log

